In [1]:
import os

In [88]:
import numpy as p
import pandas as pd

In [89]:
#changing the working directory and replacing it with where the dataset is stored 
os.chdir('D:/ml/ml-latest-small')

In [90]:
ratings_data=pd.read_csv("ratings.csv")
ratings_data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [91]:
movie_names=pd.read_csv("movies.csv")
movie_names.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [92]:
movie_data=pd.merge(ratings_data, movie_names, on='movieId')
movie_data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [93]:
movie_data.groupby('title')['rating'].mean().head()

title
'71 (2014)                                 4.0
'Hellboy': The Seeds of Creation (2004)    4.0
'Round Midnight (1986)                     3.5
'Salem's Lot (2004)                        5.0
'Til There Was You (1997)                  4.0
Name: rating, dtype: float64

In [94]:
movie_data.groupby('title')['rating'].mean().sort_values(ascending=False).head()

title
Karlson Returns (1970)                           5.0
Winter in Prostokvashino (1984)                  5.0
My Love (2006)                                   5.0
Sorority House Massacre II (1990)                5.0
Winnie the Pooh and the Day of Concern (1972)    5.0
Name: rating, dtype: float64

In [95]:
movie_data.groupby('title')['rating'].count().sort_values(ascending=False).head()

title
Forrest Gump (1994)                 329
Shawshank Redemption, The (1994)    317
Pulp Fiction (1994)                 307
Silence of the Lambs, The (1991)    279
Matrix, The (1999)                  278
Name: rating, dtype: int64

### Building a user-user collaborative filtering recommendation system

In [21]:
#conda install -c conda-forge scikit-surprise

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [3]:
#load the movielens-100k dataset 
data=Dataset.load_builtin('ml-100k')
trainset, testset=train_test_split(data, test_size=0.15) 

### User based model

In [81]:
#use user_based true/false to switch between user-based or item-based collaborative filtering
algo=KNNWithMeans(k=10, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


#we can now query for specific predictions
uid=str(196) #raw user id
iid=str(302) #raw item id

#get a prediction for specific users and items
pred=algo.predict(uid, iid, verbose=True)

In [82]:
#run the trained model against the testset
test_pred=algo.test(testset)
test_pred

[Prediction(uid='892', iid='649', r_ui=5.0, est=3.830143633932461, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='927', iid='722', r_ui=3.0, est=3.1424923429996015, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='554', iid='133', r_ui=4.0, est=4.158987347217685, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='294', iid='117', r_ui=4.0, est=3.536808397419353, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='697', iid='331', r_ui=3.0, est=3.9176913878340183, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='279', iid='489', r_ui=2.0, est=3.9552531275079987, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='416', iid='38', r_ui=3.0, est=3.0544707498101342, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='593', iid='174', r_ui=4.0, est=4.55148686895647, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='391', iid='58', r_ui=4.0, est

In [83]:
#get RMSE
print("User-based model: test set")
accuracy.rmse(test_pred, verbose=True)

User-based model: test set
RMSE: 0.9363


0.9362516460996042

### Item based model

In [84]:
#use item_based  collaborative filtering
algo=KNNWithMeans(k=10, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [85]:
#run the trained model against the testset
test_pred=algo.test(testset)
test_pred

[Prediction(uid='892', iid='649', r_ui=5.0, est=3.81495131150539, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='927', iid='722', r_ui=3.0, est=3.2650599827529687, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='554', iid='133', r_ui=4.0, est=4.157239274214993, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='294', iid='117', r_ui=4.0, est=3.8719821941276247, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='697', iid='331', r_ui=3.0, est=4.032868543005063, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='279', iid='489', r_ui=2.0, est=3.325201769138183, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='416', iid='38', r_ui=3.0, est=3.598631555200117, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='593', iid='174', r_ui=4.0, est=4.051061869465721, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='391', iid='58', r_ui=4.0, est=3

In [86]:
#get RMSE
print("Item-based model: test set")
accuracy.rmse(test_pred, verbose=True)

Item-based model: test set
RMSE: 0.9189


0.918885604701123

### Cosine 

In [113]:
#user_based
algo=KNNWithMeans(k=50, sim_options={'name': 'cosine', 'user_based': True})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [114]:
test_pred=algo.test(testset)
test_pred

[Prediction(uid='858', iid='181', r_ui=2.0, est=4.037023294314704, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='60', iid='28', r_ui=5.0, est=4.155511834281127, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='707', iid='506', r_ui=2.0, est=3.7502454538998524, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='284', iid='903', r_ui=4.0, est=3.236393718087856, details={'actual_k': 15, 'was_impossible': False}),
 Prediction(uid='332', iid='159', r_ui=5.0, est=3.9211931923570726, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='936', iid='1023', r_ui=2.0, est=3.00713529615019, details={'actual_k': 27, 'was_impossible': False}),
 Prediction(uid='867', iid='690', r_ui=5.0, est=4.046288887029614, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='534', iid='235', r_ui=4.0, est=3.338834358256361, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='347', iid='210', r_ui=4.0, est=

In [115]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9474


0.9474430481974704

In [125]:
#item based
algo=KNNWithMeans(k=50, sim_options={'name': 'cosine', 'user_based': False})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [126]:
test_pred=algo.test(testset)
test_pred

[Prediction(uid='858', iid='181', r_ui=2.0, est=4.014786209138597, details={'actual_k': 17, 'was_impossible': False}),
 Prediction(uid='60', iid='28', r_ui=5.0, est=4.090483791679022, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='707', iid='506', r_ui=2.0, est=3.7689833383302442, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='284', iid='903', r_ui=4.0, est=2.939671783962838, details={'actual_k': 38, 'was_impossible': False}),
 Prediction(uid='332', iid='159', r_ui=5.0, est=4.029708349508429, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='936', iid='1023', r_ui=2.0, est=2.944196437229734, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='867', iid='690', r_ui=5.0, est=3.6460753154706786, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='534', iid='235', r_ui=4.0, est=3.5032572504954023, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='347', iid='210', r_ui=4.0, es

In [127]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9323


0.9323238097657073